In [1]:
import cv2
print(cv2.__version__)

4.1.1-dev


In [2]:
import numpy as np
import itertools
import random
from math import log
import glob
import os.path as osp
import os

In [3]:
np.seterr(all='raise')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

# Get Directories

In [4]:
path_source = "pictures_to_stitch_2"
images = glob.glob(osp.join(path_source, "*.jpeg"))
images.sort()
path_results = "results"

In [5]:
print(images)

['pictures_to_stitch_2\\001.jpeg', 'pictures_to_stitch_2\\002.jpeg', 'pictures_to_stitch_2\\003.jpeg']


# Functions: Downscale, Read (many) --> img(s), gray(s)

In [6]:
def downscale_img(img,scale_factor):
    '''
    Downscale image by a scale factor using INTER_AREA interpolation. 
    
    Input: img - original image, 
           scale_factor - scale factor, by which height and width will be multiplied (e.g. 50/100=0,5)
           
    Return: img_s - downscaled image
    '''
    img_s = img.copy()
    if len(img_s.shape)==3:
        height, width, depth = img_s.shape
    if len(img_s.shape)==2:
        height, width = img_s.shape
    dim_s = (int(width*scale_factor),int(height*scale_factor)) 
#     print('Original dim: {}, Scale factor: {}'.format(img_s.shape,scale_factor))
#     print('After resize dim: {}'.format(dim_s))
    img_s = cv2.resize(img_s, dim_s, interpolation = cv2.INTER_AREA)
    return img_s

In [7]:
def imread_img_gray(folder, file, scale_factor):
    '''
    Read one specific image file ("file") from a specific folder ("folder") 
    scale it by "scale_factor" (e.g. 50/100=0,5)
    and convert BGR to Gray.
    
    Input: "folder" - absolute path to folder with images,
           "file" - name of the file to read,
           "scale_factor" - scale factor, by which height and width will be multiplied (e.g. 50/100=0,5)
        
    Return: "img" - scaled image, 
            "gray" - scaled gray image
    '''
    img = cv2.imread(os.path.join(folder,file))
    img = downscale_img(img,scale_factor)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img, gray

def batch_imread_imgs_grays(files,scale_factor):
    '''
    Read many specific image files ("files") from a specific folder ("folder") 
    scale it by "scale_factor" (e.g. 50/100=0,5)
    and convert BGR to Gray.
    
    Input: "folder" - absolute path to folder with images,
           "files" - list of files' names to read,
           "scale_factor" - scale factor, by which height and width will be multiplied (e.g. 50/100=0,5)
        
    Return: "imgs" - list of scaled images, 
            "grays" - list of scaled gray images
    '''
    imgs = []
    grays = []
    for file in files:
        img = cv2.imread(os.path.join(file))
#         img = downscale_img(img,scale_factor)
        imgs.append(img)
        grays.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    return imgs, grays

# Get and Prepare imgs

In [9]:
imgs, grays = batch_imread_imgs_grays(images, 50/100)

# Out of Box stitching

stitcher = cv2.Stitcher_create()
status, result_img = stitcher.stitch(imgs)

if status == 0:
    cv2.imwrite(os.path.join(absolute_path_results,'out_of_box','pano_sift.jpg'),result_img)
else:
    print("Status: {}".format(status))
for n, img in enumerate(imgs):
    cv2.imwrite(os.path.join(absolute_path_results,'out_of_box','img{}_s.jpg'.format(n)),img)

# Create Detectors

## SIFT detector

In [10]:
sift = cv2.xfeatures2d.SIFT_create(500) # limit feature points to 500

## SURF detector

In [11]:
surf = cv2.xfeatures2d.SURF_create() # Hessian threshold

## ORB detector

In [12]:
orb = cv2.ORB_create(nfeatures=500) # limit feature points to 500

# Get KeyPoints and Descriptors

In [13]:
def get_kp_des(gray, detector):
    kp, des = detector.detectAndCompute(gray, None)
    return kp, des

# MAIN PROCESS

## Get KeyPoints & Descriptors

In [14]:
detector = sift

In [15]:
kp_0, des_0 = get_kp_des(grays[1], detector)
kp_1, des_1 = get_kp_des(grays[2], detector)

In [16]:
des_0.shape

(500, 128)

In [17]:
des_1.shape

(500, 128)

In [18]:
img0_kp = imgs[0].copy()
img0_kp = cv2.drawKeypoints(img0_kp,kp_0,img0_kp, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite(os.path.join(path_results,'key_points','img000_sift.jpg'),img0_kp)

True

In [19]:
img1_kp = imgs[1].copy()
img1_kp = cv2.drawKeypoints(img1_kp,kp_1,img1_kp, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite(os.path.join(path_results,'key_points','img001_sift.jpg'),img1_kp)

True

# Use Lowe criteria for points matching
A robust criteria (also introduced by Lowe) for matching a feature in one image to a
feature in another image is to use the ratio of the distance to the two closest matching
features. This ensures that only features that are distinct enough compared to the
other features in the image are used. As a consequence, the number of false matches
is lowered.

In [20]:
def match(des_0, des_1, dist_ratio = 0.7):
    """ 
    For each descriptor in the first image, select its match in the second image.
    Idea - use the angle between descriptor vectors (normalised) as distance measure.
    
    Input: 
        "des_0" - descriptors for the first image
        "des_1" - same for second image 
    
    Return: 
        "matchscores" - list of indexes for points from the 2nd picture for every position (kp number) in first
        e.g: matchscores[0] = 1 means that for the first key point in the 1st image the closest is second in the 2nd image
    """
    
    # des_0 = np.array(des_0, dtype=np.float64)
    # des_1 = np.array(des_1, dtype=np.float64)
    
    des_0_size = des_0.shape[0]
    
    matchscores = [-1] * des_0_size
    
    # des_1_T = des_1.T # precompute matrix transpose
    
    # for i in range(des_0_size[0]):  
    #     dotprods = np.dot(des_0[i,:],des_1_T) # vector of dot products
    #     dotprods = 0.9999*dotprods
    #     # inverse cosine and sort ascending, return index for features in second image 
    #     # https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html
    #     indx = np.argsort(np.arccos(dotprods))
    #     # check if nearest neighbor has angle less than dist_ratio times 2nd
    #     if np.arccos(dotprods)[indx[0]] < dist_ratio * np.arccos(dotprods)[indx[1]]:
    #         matchscores[i] = int(indx[0])
    for i, p in enumerate(des_0):
        distances = list(map(lambda x: np.linalg.norm(x - p), des_1))
        idx = np.argsort(distances)
        if distances[idx[0]] < dist_ratio * distances[idx[1]]:
            matchscores[i] = idx[0]
    return matchscores

# SKIPPED SECOND MATCH ON PURPOSE

In [21]:
def match_twosided(des_0,des_1):
    
    """ Two-sided symmetric version of match() - see above """
    
    matches_1_to_2 = match(des_0, des_1)
    # matches_2_to_1 = match(des_1, des_0)
    
    # print(matches_1_to_2.reshape((matches_1_to_2.shape[0],)))
    # print(matches_2_to_1.reshape((matches_2_to_1.shape[0],)))
    # remove matches that are not symmetric
    # for i, n in enumerate(matches_1_to_2):
    #     if n == -1:
    #         continue
    #     if matches_2_to_1[n] != i:
    #         matches_1_to_2[i] = -1
    
    return matches_1_to_2

In [22]:
matches_1_to_2 = match_twosided(des_0,des_1)

In [23]:
matches_1_to_2

[-1,
 175,
 -1,
 -1,
 -1,
 210,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 114,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 230,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 231,
 -1,
 119,
 -1,
 -1,
 232,
 -1,
 -1,
 -1,
 -1,
 235,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 340,
 -1,
 340,
 254,
 -1,
 -1,
 -1,
 255,
 256,
 -1,
 -1,
 258,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 327,
 98,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 273,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 285,
 -1,
 286,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 180,
 183,
 -1,
 -1,
 -1,
 -1,
 -1,
 126,
 -1,
 -1,
 -1,
 122,
 -1,
 -1,
 -1,
 -1,
 179,
 -1,
 -1,
 -1,
 -1,
 268,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 268,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,


In [24]:
def matches_to_mutual_corr_list(matches_1_to_2):
    ''' Adapt for RANSAC fuction format: [# from 1st, # from 2nd image]'''
    mutual_corr_list = []
    for indx, match in enumerate(matches_1_to_2):
        if match != -1:
            mutual_corr_list.append([indx, match])  #[key point from 1st image, same from 2nd]
    return mutual_corr_list

In [25]:
mutual_corr_list = matches_to_mutual_corr_list(matches_1_to_2)

In [26]:
mutual_corr_list

[[1, 175],
 [5, 210],
 [19, 114],
 [53, 230],
 [61, 231],
 [63, 119],
 [66, 232],
 [71, 235],
 [87, 340],
 [89, 340],
 [90, 254],
 [94, 255],
 [95, 256],
 [98, 258],
 [106, 327],
 [107, 98],
 [117, 273],
 [131, 285],
 [133, 286],
 [159, 180],
 [160, 183],
 [166, 126],
 [170, 122],
 [175, 179],
 [180, 268],
 [188, 268],
 [195, 156],
 [196, 162],
 [201, 137],
 [206, 86],
 [208, 85],
 [209, 217],
 [216, 123],
 [219, 228],
 [225, 122],
 [226, 241],
 [233, 98],
 [235, 259],
 [240, 257],
 [243, 470],
 [250, 195],
 [258, 30],
 [259, 158],
 [276, 72],
 [284, 194],
 [285, 193],
 [294, 84],
 [297, 364],
 [298, 288],
 [299, 288],
 [300, 296],
 [310, 436],
 [312, 0],
 [324, 26],
 [356, 38],
 [363, 283],
 [370, 29],
 [378, 270],
 [384, 59],
 [385, 270],
 [387, 31],
 [390, 289],
 [393, 334],
 [394, 160],
 [399, 159],
 [408, 253],
 [418, 265],
 [426, 149],
 [427, 244],
 [429, 244],
 [451, 154],
 [460, 245],
 [465, 364],
 [466, 310],
 [473, 266],
 [476, 187],
 [477, 185],
 [480, 325],
 [481, 234],
 [4

# Check collinearity of points

Check if 3 points are collinear - if true: can't be used to calculate homography matrix.

In [27]:
def if_2_vectors_collinear(vec1,vec2,eps = 1e-6):
    if_collinear = False
    vec1_norm = np.linalg.norm(vec1)
    vec2_norm = np.linalg.norm(vec2)
    if vec1_norm>eps and vec2_norm>eps: 
        vec_dot = np.dot(vec1,vec2)/vec1_norm/vec2_norm
        if abs(vec_dot-1)<=eps:
            if_collinear = True
    return if_collinear

In [28]:
def if_3collinear_in_set(sample_set, kp, eps = 1e-6):
    '''
    check if any 3 points in a set (e.g. 4 points for homograpy) is collinear
    which is inappropriate to find homography matrix
    
    Through dot product for 2 dimensional.
    ps: cos(0 grad) = 1
    eps (by def) = 10^(-6)
    
    Input:
        'sample_set' - 4 points from 1 image to find homography matrix
        'kp' - key points from set
        'eps' - accuracy, 10^(-6) by default
    
    Return:
        'if_collinear' - boolean if there is a 3 point on one line
    '''
    
    # get points from kp
    points = []
    for point_num in sample_set:
        points.append(kp[point_num])
    
    # for each combination of 3 points find collinear
    if_collinear = False
    for three_points in itertools.combinations(points,3):
        vec1 = np.array(three_points[1].pt)-np.array(three_points[0].pt)
        vec2 = np.array(three_points[2].pt)-np.array(three_points[0].pt)
        
        if_collinear = if_2_vectors_collinear(vec1,vec2,eps)
        if if_collinear:
            break
    return if_collinear

# Determine ERROR measurement

In [29]:
def dist_by_angle(vec_0,vec_1): 
    
    ''' If we use distance as angle between 2 vectors '''

    vec_0_norm = vec_0/np.linalg.norm(vec_0)
    vec_1_norm = vec_1/np.linalg.norm(vec_1)
    
    cross_prod = np.dot(vec_0_norm,vec_1_norm.T)
    cross_prod = 0.9999*cross_prod
    
    distance_angle = np.arccos(cross_prod)
    
    return distance_angle

In [30]:
def get_total_error(h,sample,kp_0,kp_1,mutual_corr_list,dist = 'angle'):
    '''
    Calculate total error for all points based on given homography matrix
    "dist" - must be "angle" or "norm"
    '''
    assert dist in ["angle","norm"],("Expected 'angle' or 'norm'")
    distances = []
    for i,j in mutual_corr_list:
        kp_0_h = np.array((kp_0[i].pt[0],kp_0[i].pt[1],1), dtype = "float64").dot(h)
        kp_0_h = np.array((kp_0_h[0]/kp_0_h[2],kp_0_h[1]/kp_0_h[2],1), dtype = "float64")
        kp_1_orig = np.array((kp_1[j].pt[0],kp_1[j].pt[1],1), dtype = "float64")
        
        if dist == 'norm':
            distance = cv2.norm(kp_0_h,kp_1_orig)
        else:
            distance = dist_by_angle(kp_0_h,kp_1_orig)
        
        distances.append(distance)
        
    return sum(distances)

In [31]:
def get_keypoints_pt_by_sample(kp_0,kp_1,sample):
    '''
    Given list of points, return list of their coordinates to calculate homography matrix
    '''
    kp_0_sample=[]
    kp_1_sample=[]
    for i,j in sample:
        kp_0_sample.append(kp_0[i].pt)
        kp_1_sample.append(kp_1[j].pt)
    return kp_0_sample, kp_1_sample

# RANSAC

Random Sampling Consensus

1. Randomly choose 4 points in the first image and 4 point in the second image (avoid using same points and use mutual_corr_list to choose from) 
1. Calculate Homography matrix
1. Calculate inliers
1. Repeat until needed accuracy and precision are reached

In [32]:
def get_4_different_pairs(mutual_corr_list):
    sample_set_1 = set()
    sample_set_2 = set()
    
    while True:
        sample = random.sample(mutual_corr_list,4)
        sample_set_1.clear()
        sample_set_2.clear()
        for i,j in sample:
            sample_set_1.add(i)
            sample_set_2.add(j)
        if len(sample_set_1) ==4 and len(sample_set_2)==4: 
            if not if_3collinear_in_set(sample_set_1, kp_0) and not if_3collinear_in_set(sample_set_2, kp_1):
                break

    return sample

In [33]:
def get_ransac_req_iter_num(e=0.5,s=4,p=0.99):
    #e - probability that a point is an outlier
    #s - number of points in the sample: 4 is for homography
    #p - desired probability that we got good sample
    iter_num_required = log(1-p)/log(1-(1-e)**s)
    req_iter = int(iter_num_required)+1
    return req_iter

def get_inliers(h, sample, kp_0, kp_1, mutual_corr_list, threshold, dist):
    '''
    calculate inliers for points not in sample, but can I calculate and for the sample points?
    '''
    appropriate = []
    for i,j in mutual_corr_list:
        kp_1_h = h.dot(np.array((kp_1[j].pt[0], kp_1[j].pt[1], 1), dtype = "float64"))
        kp_1_h /= kp_1_h[2]
        kp_0_orig = np.array((kp_0[i].pt[0], kp_0[i].pt[1], 1), dtype = "float64")
        
        if dist == 'norm':
            distance = cv2.norm(kp_1_h - kp_0_orig)
        else:
            distance = dist_by_angle(kp_0_h,kp_1_orig)
        
        if distance <= threshold:
            appropriate.append(distance)
    return appropriate

In [34]:
def ransac_find_homography_matrix(kp_0, kp_1, mutual_corr_list,threshold, iter_num_required=0,
                                  dist = 'norm', inliers_error = 'inliers'):
    '''
    0) calculate required number of iterations
    repeat for required number of iterations:
    1) get sample of kps
    2) calculate Homography matrix
    3) calculate inliers|total error
    4) save the matrix with higher number of matchinliers|lower total error
    '''
    # 0
    if iter_num_required == 0:
        iter_num_required = get_ransac_req_iter_num()
    # print(kp_0)
    # kp_0 = np.array([kp_0[i].pt for i, _ in mutual_corr_list])
    # kp_1 = np.array([kp_1[i].pt for _, i in mutual_corr_list])
    # best_homography, _ = cv2.findHomography(kp_1, kp_0, cv2.RANSAC, threshold)

    best_homography = []
    max_inliers_num = 0 
    total_error_min = -1
    for x in range(iter_num_required):
        # 1
        sample = get_4_different_pairs(mutual_corr_list)
        # print('Sample is: {}'.format(sample))
        kp_0_sample, kp_1_sample = get_keypoints_pt_by_sample(kp_0,kp_1,sample)
        # 2
        h, status = cv2.findHomography(np.array(kp_1_sample), np.array(kp_0_sample), 10**6)
        
        # 3
        if inliers_error == 'inliers':
            inliers_dist = get_inliers(h,sample,kp_0,kp_1,mutual_corr_list,threshold, dist)
            curr_inliers_num = len(inliers_dist)
            if max_inliers_num < curr_inliers_num:
                print(f'Change of model!!!\nPrevious max_inliers_num: {max_inliers_num} \nCurrent curr_inliers_num: {curr_inliers_num}')
                max_inliers_num = curr_inliers_num
                best_homography = h
        
    #     if inliers_error == 'error':
    #         total_error = get_total_error(h,sample,kp_0,kp_1,mutual_corr_list, dist)
    #         # print(f'total_error = {total_error}')
    #         if total_error_min ==-1:
    #             total_error_min = total_error
    #         else:
    #             if total_error_min > total_error:
    #                 total_error_min = total_error
    #                 best_homography = h
        
    print(f'Maximum inliers: {max_inliers_num}',f'Total error min: {total_error_min}', f'Best homography: {best_homography}', sep = '\n')
    return best_homography, total_error_min

In [35]:
threshold = 4

In [36]:
h, error = ransac_find_homography_matrix(kp_0, kp_1, mutual_corr_list, threshold, iter_num_required=0, dist = "norm",inliers_error='inliers')

Change of model!!!
Previous max_inliers_num: 0 
Current curr_inliers_num: 8
Change of model!!!
Previous max_inliers_num: 8 
Current curr_inliers_num: 13
Change of model!!!
Previous max_inliers_num: 13 
Current curr_inliers_num: 14
Change of model!!!
Previous max_inliers_num: 14 
Current curr_inliers_num: 45
Maximum inliers: 45
Total error min: -1
Best homography: [[ 8.28440011e-01 -3.15950215e-02  4.03062325e+02]
 [-1.81552537e-01  9.87680882e-01 -1.12336252e+01]
 [-2.97831913e-04  1.22085370e-05  1.00000000e+00]]


In [39]:
if np.all(h) != 0: 
    img_1_after_homography = imgs[2].copy()
    result = cv2.warpPerspective(img_1_after_homography, h, (imgs[0].shape[1]+imgs[1].shape[1],imgs[1].shape[0]))
    coco=cv2.imwrite(os.path.join(path_results,'warped',f'{path_source}_img000_warped_before_sift.jpg'),result)
    result[:imgs[1].shape[0],:imgs[1].shape[1]]=imgs[1]
    coco2=cv2.imwrite(os.path.join(path_results,'warped',f'{path_source}_img000_warped_after_sift.jpg'),result)

if np.all(h) != 0: 
    img_0_after_homography = imgs[0].copy()
    img_0_after_homography = cv2.warpPerspective(img_0_after_homography, h, (imgs[1].shape[1],imgs[1].shape[0]))

    cv2.imwrite(os.path.join(path_results,'warped',f'{path_source}_img000_warped_sift_{error}.jpg'),img_0_after_homography)